# King County House Price Prediction

This notebook was created for the IBM meetup on February 13 2020. It contains neural networks trained to predict house prices in King County, Washington, USA based on features of the house. They are trained using different preprocessing methods like PCA and Standardization. They are also trained using different Neural Networks with one featuring batch normalization. We will explore the benefits of data preprocessing on our neural networks and how they affect performance.

## Loading the Data, Creating NN, and Training Function

Importing the necessary libraries.

In [ ]:
pip install Pillow==6.2.2

***Restart the kernel using the options on the top left***

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from sklearn.decomposition import PCA
from torch.utils.data import Dataset
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.nn.functional as F
import matplotlib.pylab as plt
torch.manual_seed(1)
np.random.seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

Here we load the dataset we will use and the already trained models to save time instead of training from scratch.

In [ ]:
!wget https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/meet_up/12.02.2020/king_county_house_data.csv
!wget https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/meet_up/12.02.2020/pca_model.pt
!wget https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/meet_up/12.02.2020/regular_model.pt
!wget https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/meet_up/12.02.2020/standardized_model.pt

Turn the csv into a dataframe

In [ ]:
df = pd.read_csv('king_county_house_data.csv')
print(df.shape)
df.head()

Create a PyTorch Dataset object to store our dataframe values in. We create 3 type of data one for data with PCA, Standardization, and regular data with no preprocessing.

In [ ]:
class data(Dataset):
    
    def __init__(self, test=False, regular=False, pca=False, standard=False):
        
        X = df.drop(columns='price')
        Y = df['price']
        # Split the data into training data and testing data using a 80/20 split
        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=1)
        
        x_train = x_train.astype(float)
        x_test = x_test.astype(float)
        
        if (regular):
            
            if (test):
                self.x = torch.FloatTensor(x_test.values)
                self.y = torch.FloatTensor(y_test.values).view(-1,1)
            else:
                self.x = torch.FloatTensor(x_train.values)
                self.y = torch.FloatTensor(y_train.values).view(-1,1)
                
        elif (pca):
            
            # Performs PCA on the data
            pca = PCA(whiten=True)
            x_train = pca.fit_transform(x_train)
            x_test = pca.transform(x_test)
            
            if (test):
                self.x = torch.FloatTensor(x_test)
                self.y = torch.FloatTensor(y_test.values).view(-1,1)
            else:
                self.x = torch.FloatTensor(x_train)
                self.y = torch.FloatTensor(y_train.values).view(-1,1)
                
        else:
            
            # Performs Standardization on the data
            standard = preprocessing.StandardScaler()
            x_train = standard.fit_transform(x_train)
            x_test = standard.transform(x_test)
            
            if (test):
                self.x = torch.FloatTensor(x_test)
                self.y = torch.FloatTensor(y_test.values).view(-1,1)
            else:
                self.x = torch.FloatTensor(x_train)
                self.y = torch.FloatTensor(y_train.values).view(-1,1)

    # Getter
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

Here we define the Neural Networks we will use. They are both 4 layered networks with 1 input layer, 1 output layer, and 2 hidden layers. THe number of nodes in each layer is 2/3 of the input layer except the output because we are doing regression. They both use relu as the activation function.

The difference is that one uses Batch Normalization and one does not.

In [ ]:
class BatchNormalizationNN(nn.Module):
    
    # Constructor
    def __init__(self):
        super(BatchNormalizationNN, self).__init__()
        # The dataset has 18 features
        self.linear1 = nn.Linear(18, 12)
        self.b1 = nn.BatchNorm1d(12)
        self.linear2 = nn.Linear(12, 8)
        self.b2 = nn.BatchNorm1d(8)
        self.linear3 = nn.Linear(8, 1)

    # Prediction    
    def forward(self, x): 
        y = self.b1(torch.relu(self.linear1(x)))
        y = self.b2(torch.relu(self.linear2(y)))
        y = self.linear3(y)
        return y

In [ ]:
class RegularNN(nn.Module):
    
    # Constructor
    def __init__(self):
        super(RegularNN, self).__init__()
        # The dataset has 18 features
        self.linear1 = nn.Linear(18, 12)
        self.linear2 = nn.Linear(12, 8)
        self.linear3 = nn.Linear(8, 1)

    # Prediction    
    def forward(self, x): 
        y = torch.relu(self.linear1(x)) 
        y = torch.relu(self.linear2(y))
        y = self.linear3(y)
        return y

Function performs the training of the models. It is given the training data, testing data, criterion (loss function), model(Neural Network), optimizer, and epochs (Number of times the model is trained on the entire dataset). 

In [ ]:
def train(train_loader, test_loader, criterion, model, optimizer, epochs):
    
    train_loss = []
    test_loss = []
    
    for epoch in range(epochs):
        
        print(epoch)
        
        total_loss = 0
        
        for i, (x, y) in enumerate(train_loader):
            # Set the model in training mode so it updates values in Batch Normalization
            model.train()
            # Clears the optimizer
            optimizer.zero_grad()
            # Calculates the prediction the model gives
            z = model(x)
            # Calculates the loss between the prediction and the actual value
            loss = criterion(z, y)
            # Calcualates the partial deriviative for each parameter of the Neural Network
            loss.backward()
            # Adjusts the parameters according to the optimizer
            optimizer.step()
            total_loss = total_loss + loss.item()
        
        train_loss.append(total_loss/(len(train_loader.dataset)/train_loader.batch_size))
        print(total_loss/(len(train_loader.dataset)/train_loader.batch_size))
        
        total_loss = 0
        
        for i, (x, y) in enumerate(test_loader):
            # Set the model in evaluation mode so we do not train the Batch Normalization while evaluating our model
            model.eval()
            z = model(x)
            loss = criterion(z, y)
            total_loss = total_loss + loss.item()
           
        test_loss.append(total_loss/len(test_loader.dataset))
        print(total_loss/len(test_loader.dataset))
        
    return train_loss, test_loss

## Loading the Pre-Trained Models

In [ ]:
RegularModel = RegularNN()
checkpoint = torch.load('regular_model.pt')
RegularModel.load_state_dict(checkpoint['model_state_dict'])
RegularModel.eval()
regular_train_loss = checkpoint['train_loss']
regular_test_loss = checkpoint['test_loss']

In [ ]:
PCAModel = BatchNormalizationNN()
checkpoint = torch.load('pca_model.pt')
PCAModel.load_state_dict(checkpoint['model_state_dict'])
PCAModel.eval()
pca_train_loss = checkpoint['train_loss']
pca_test_loss = checkpoint['test_loss']

In [ ]:
StandardModel = BatchNormalizationNN()
checkpoint = torch.load('standardized_model.pt')
StandardModel.load_state_dict(checkpoint['model_state_dict'])
StandardModel.eval()
stan_train_loss = checkpoint['train_loss']
stan_test_loss = checkpoint['test_loss']

## Training

The code in the sections below is code to train the models with each type of preprocessing. The training code is saved a raw so it does not accidentally get run as it would take a lot of time to train. The loss function used is SmoothL1Loss and in its current setup it gives the absolute value of the error between the prediction and actual value. The optimizer used is Adadelta, we use this because we do not need to worry about initializing or updating the rate at which the Neural Network learns. If you would like to experiment with the Neural Networks please change the type of the training block to code using the options at the top. 

## Regular NN

In [ ]:
plt.plot(regular_train_loss)
plt.title("Regular Train")
plt.xlabel("Epochs")
plt.ylabel("Average Loss ")
plt.show()
print(regular_train_loss[-1])

In [ ]:
plt.plot(regular_test_loss)
plt.title("Regular Test")
plt.xlabel("Epochs")
plt.ylabel("Average Loss ")
plt.show()
print(regular_test_loss[-1])

## PCA NN

In [ ]:
plt.plot(pca_train_loss)
plt.title("PCA Train")
plt.xlabel("Epochs")
plt.ylabel("Average Loss")
plt.show()
print(pca_train_loss[-1])

In [ ]:
plt.plot(pca_test_loss)
plt.title("PCA Test")
plt.xlabel("Epochs")
plt.ylabel("Average Loss")
plt.show()
print(pca_test_loss[-1])

## Standardized NN

In [ ]:
plt.plot(stan_train_loss)
plt.title("Standardization Train")
plt.xlabel("Epochs")
plt.ylabel("Average Loss")
plt.show()
print(stan_train_loss[-1])

In [ ]:
plt.plot(stan_test_loss)
plt.title("Standardization Test")
plt.xlabel("Epochs")
plt.ylabel("Average Loss")
plt.show()
print(stan_test_loss[-1])

## Viewing the Results

The code below will allow you to see the predicted price, actual price, and error for each house in the test dataset.

In [ ]:
criterion = nn.SmoothL1Loss()
limit = 9

In [ ]:
d_test = data(regular=True, test=True)
test_loader = torch.utils.data.DataLoader(dataset=d_test)

for i, (x, y) in enumerate(test_loader):
    RegularModel.eval()
    z = RegularModel(x)
    loss = criterion(z, y)
    print(i)
    print('Predicted: ',z.item())
    print('Actual: ', y.item())
    print('Loss: ', loss.item())
    if (i == limit):
        break

In [ ]:
d_test = data(pca=True, test=True)
test_loader = torch.utils.data.DataLoader(dataset=d_test)

for i, (x, y) in enumerate(test_loader):
    PCAModel.eval()
    z = PCAModel(x)
    loss = criterion(z, y)
    print(i)
    print('Predicted: ',z.item())
    print('Actual: ', y.item())
    print('Loss: ', loss.item())
    if (i == limit):
        break

In [ ]:
d_test = data(standard=True, test=True)
test_loader = torch.utils.data.DataLoader(dataset=d_test)

for i, (x, y) in enumerate(test_loader):
    StandardModel.eval()
    z = StandardModel(x)
    loss = criterion(z, y)
    print(i)
    print('Predicted: ',z.item())
    print('Actual: ', y.item())
    print('Loss: ', loss.item())
    if (i == limit):
        break

Created By: Azim Hirjani